In [1]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

In [2]:
raw_csv_data = np.loadtxt('audiobooks_data.csv', delimiter=',')
unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:, -1] # last col

# balancing 0s and 1s
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]): # length of vector
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [3]:
# standardize inputs
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# shuffle
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

# train validation test
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - validation_samples_count - train_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

np.savez('audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('audiobooks_data_test', inputs=test_inputs, targets=test_targets)

1786.0 3579 0.4990220732048058
215.0 447 0.4809843400447427
236.0 448 0.5267857142857143


In [4]:
npz = np.load('audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

npz = np.load('audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(float)
validation_targets = npz['targets'].astype(int)

npz = np.load('audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(float)
test_targets = npz['targets'].astype(int)

In [5]:
input_size = 10
output_size = 2
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

batch_size = 32
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

model.fit(train_inputs, train_targets, batch_size=batch_size, epochs=max_epochs,
          callbacks=[early_stopping],
         validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/100


2025-09-19 20:34:37.226268: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-19 20:34:37.226293: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-19 20:34:37.226299: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-19 20:34:37.226312: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-19 20:34:37.226320: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-09-19 20:34:37.513118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


112/112 - 1s - 13ms/step - accuracy: 0.7329 - loss: 0.4897 - val_accuracy: 0.7763 - val_loss: 0.4082
Epoch 2/100
112/112 - 1s - 6ms/step - accuracy: 0.7639 - loss: 0.4329 - val_accuracy: 0.7897 - val_loss: 0.3788
Epoch 3/100
112/112 - 1s - 6ms/step - accuracy: 0.7639 - loss: 0.4157 - val_accuracy: 0.8098 - val_loss: 0.3657
Epoch 4/100
112/112 - 1s - 6ms/step - accuracy: 0.7645 - loss: 0.4093 - val_accuracy: 0.7964 - val_loss: 0.3593
Epoch 5/100
112/112 - 1s - 7ms/step - accuracy: 0.7720 - loss: 0.4012 - val_accuracy: 0.7897 - val_loss: 0.3575
Epoch 6/100
112/112 - 1s - 6ms/step - accuracy: 0.7717 - loss: 0.3963 - val_accuracy: 0.8031 - val_loss: 0.3478
Epoch 7/100
112/112 - 1s - 6ms/step - accuracy: 0.7726 - loss: 0.3945 - val_accuracy: 0.8076 - val_loss: 0.3479
Epoch 8/100
112/112 - 1s - 6ms/step - accuracy: 0.7756 - loss: 0.3892 - val_accuracy: 0.7897 - val_loss: 0.3456
Epoch 9/100
112/112 - 1s - 6ms/step - accuracy: 0.7681 - loss: 0.3900 - val_accuracy: 0.8098 - val_loss: 0.3422
Epo